In [1]:
import requests
import json
import chromadb
import openai
import pandas as pd
from tqdm import tqdm

chroma_client = chromadb.PersistentClient(path="db/")
try:
    collection = chroma_client.create_collection(name="my_collection")
except:
    collection = chroma_client.get_collection(name="my_collection")

In [2]:
client = openai.OpenAI(
    api_key="",
)


def get_embeddings(text):
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=f"{text}",
    )
    return response.data[0].embedding

In [4]:
path = ""
# load json path to pandas dataframe
df = pd.read_json(path)

In [57]:
df.columns

Index(['Chapter_Number', 'Chapter_English', 'Chapter_Arabic', 'Section_Number',
       'Section_English', 'Section_Arabic', 'Hadith_number', 'English_Hadith',
       'English_Isnad', 'English_Matn', 'Arabic_Hadith', 'Arabic_Isnad',
       'Arabic_Matn', 'Arabic_Comment', 'English_Grade', 'Arabic_Grade',
       'embeddings'],
      dtype='object')

In [58]:
df.iloc[10]["English_Hadith"]

'narrated anas bin malik once an ansari woman accompanied by a son of hers came to allahs messenger ﷺ allahs messenger ﷺ spoketo her and said twice by him in whose hand my life is you are the most beloved people to me'

In [14]:
embeddings = df["embeddings"]

In [17]:
for i in tqdm(range(len(embeddings))):
    collection.add(ids=str(i), embeddings=embeddings[i])

100%|██████████| 7345/7345 [02:32<00:00, 48.25it/s]


In [20]:
emb1 = df.iloc[0]["embeddings"]
text1 = df.iloc[0]["Arabic_Hadith"]

In [23]:
embeddings = get_embeddings(text1)

In [79]:
import torch

In [81]:
%%timeit
# sample matrix mul
random_1 = torch.randn(1, 200000)
random_2 = torch.randn(200000, 1024)

result = torch.matmul(random_1, random_2)


1.17 s ± 247 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [82]:
%%timeit
# sample matrix mul
random_1 = torch.randn(1, 200000, device = "cuda")
random_2 = torch.randn(200000, 1024, device = "cuda")

result = torch.matmul(random_1, random_2)


6.57 ms ± 27.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
arr = []
arr2 = []
arr3 = []arr3

for element1, element2 in zip(embeddings, emb1):
    if element1 > element2:
        arr3.append(element1)
    elif element1 < element2:
        arr3.append(element2)
    
arr3 = [element1 if element1 > element2 else element2 for element1, element2 in zip(embeddings, emb1)]       

In [119]:
def get_hadith(arabic_prompt, translate=True):
    # Translate into English using ChatGPT
    if translate:
        english_prompt = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "user",
                    "content": f"Translate the following Arabic text into English: {arabic_prompt}",
                },
            ],
        )
        english_prompt = english_prompt.choices[0].message.content
        print(english_prompt)
    else:
        english_prompt = arabic_prompt
    embeddings = get_embeddings(arabic_prompt)
    results = collection.query(embeddings, n_results=5)
    ids = [int(id) for id in results["ids"][0]]
    hadiths = df.iloc[ids]["Arabic_Hadith"]
    for hadith in hadiths:
        print(hadith)
        print("*" * 10)

In [123]:
get_hadith("Intentions are crucial", False)

حدثنا الحميدي عبدالله بن الزبير قال حدثنا سفيان قال حدثنا يحيى بن سعيد الأنصاري قال أخبرني محمد بن إبراهيم التيمي أنه سمع علقمة بن وقاص الليثي يقول سمعت عمر بن الخطاب ـ رضى الله عنه ـ على المنبر قال سمعت رسول الله صلى الله عليه وسلم يقول   إنما الأعمال بالنيات وإنما لكل امرئ ما نوى فمن كانت هجرته إلى دنيا يصيبها أو إلى امرأة ينكحها فهجرته إلى ما هاجر إليه 
**********
حدثنا مسدد حدثنا يحيى عن سفيان حدثني سلمة بن كهيل وحدثنا أبو نعيم حدثنا سفيان عن سلمة قال سمعت جندبا يقول قال النبي صلى الله عليه وسلم ولم أسمع أحدا يقول قال النبي صلى الله عليه وسلم غيره فدنوت منه فسمعته يقول قال النبي صلى الله عليه وسلم   من سمع سمع الله به ومن يرائي يرائي الله به 
**********
حدثنا عبد العزيز بن عبد الله حدثنا سليمان عن موسى بن عقبة عن أبي سلمة بن عبد الرحمن عن عائشة أن رسول الله صلى الله عليه وسلم قال   سددوا وقاربوا واعلموا أن لن يدخل أحدكم عمله الجنة وأن أحب الأعمال أدومها إلى الله وإن قل 
**********
حدثنا قتيبة بن سعيد حدثنا عبد الوهاب قال سمعت يحيى بن سعيد يقول أخبرني محمد بن إبراهيم أنه سمع علقمة ب